In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd


In [2]:



def solve_hotelling3(phi, lambd):
    n = 3
    A = np.eye(n)
    for i in range(n):
        A[i, (i-1)%n] -= 0.5
        A[i, (i+1)%n] -= 0.5
    b = np.zeros(n)
    for i in range(n):
        b[i] = phi[i] - 0.5*(phi[(i-1)%n] + phi[(i+1)%n]) + lambd
    A_full = np.vstack([A, np.ones(n)])
    b_full = np.append(b, np.mean(phi) + lambd/3)
    price = np.linalg.lstsq(A_full, b_full, rcond=None)[0]
    return price


def solve_hotelling2(phi, lambd, idx):
    i, j = idx
    p_i = (2 * phi[i] + phi[j]) / 3 + lambd
    p_j = (2 * phi[j] + phi[i]) / 3 + lambd
    prices = np.zeros(len(phi))
    prices[i] = p_i
    prices[j] = p_j
    return prices


def calc_price(phi, lambd, wtp):
    phi = np.array(phi)
    n = len(phi)
    price = np.zeros(n)
    eps = 1e-8

    if n == 1:
        price = np.array([max(phi[0], min(wtp, wtp-lambd))])
    elif n == 2:
        price = solve_hotelling2(phi, lambd, [0, 1])
    elif n == 3:
        # All equal? Symmetric solution
        if np.max(phi) - np.min(phi) < eps:
            price = np.ones(n) * (phi[0] + lambd/3)
        else:
            # Try general solution
            price_try = solve_hotelling3(phi, lambd)
            ok = (price_try >= phi - eps) & (price_try <= wtp + eps)
            if np.all(ok):
                price = price_try
            else:
                # Find two lowest-cost firms, use 2-firm solution
                active = np.argsort(phi)[:2]
                price = solve_hotelling2(phi, lambd, active)
    else:
        raise NotImplementedError("Only supports n=1, 2, or 3.")
    return price

    
def shares_hotelling3(prices, cost):
    n = len(prices)
    s = np.zeros(n)
    circumference = 1.0
    for i in range(n):
        left = (i-1) % n
        right = (i+1) % n
        arc = circumference / n
        delta_p_left = prices[i] - prices[left]
        z_left = (delta_p_left / (2*cost)) + (arc/2)
        delta_p_right = prices[right] - prices[i]
        z_right = (delta_p_right / (2*cost)) + (arc/2)
        s[i] = z_left + z_right
    s *= (circumference / np.sum(s))  # normalize
    return s    


def shares_hotelling2(prices, cost, idx):
    # idx: indices of two active firms
    i, j = idx
    p_i, p_j = prices[i], prices[j]
    s_i = 0.5 + (p_j - p_i) / (2 * cost)
    s_j = 1.0 - s_i
    s = np.zeros(len(prices))
    s[i], s[j] = s_i, s_j
    return s
    
    
def calc_s(phi, cost, wtp, circle=True):
    phi = np.array(phi)
    n = len(phi)
    prices = calc_price(phi, cost, wtp)

    if np.all(prices == 0):
        return np.zeros(n)

    active = np.where(prices > 0)[0]

    if len(active) == 1:
        s = np.zeros(n)
        s[active[0]] = 1.0
        return s
    elif len(active) == 2:
        return shares_hotelling2(prices, cost, active)
    elif len(active) == 3:
        # All active; circle
        s = shares_hotelling3(prices, cost)
        # If crazy numbers (e.g., shares < 0), fallback to zeros.
        if np.any(s < 0) or np.any(s > 1):
            return np.zeros(n)
        else:
            return s
    else:
        return np.zeros(n)


def calc_profits(phi, cost, wtp, mc):
    s = calc_s(phi, cost, wtp)
    prices = calc_price(phi, cost, wtp)
    hospital_profit = np.sum(s * (phi - mc))
    insurer_profits = s * (prices - phi)
    return hospital_profit, insurer_profits, s, prices


def analytic_phi(cost, wtp, n, belief="active"):
    if n==0:
        return 0
    if n == 1:
        return (wtp - cost) / 2
    elif n == 2:
        if belief == "active":
            return 0.75*cost + (wtp-cost)/2
        elif belief == "passive":
            return 1.5*cost
    elif n == 3:
        if belief == "active":
            return cost + wtp/2
        elif belief == "passive":
            return 9/4*cost
    return None


def nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement=0):
    hospital_profit, insurer_profits, s, prices = calc_profits(phi_vec, cost, wtp, mc)
    #print(phi_vec, hospital_profit, disagreement, insurer_profits, s, prices)
    if hospital_profit - disagreement <= 1e-8 or insurer_profits[0] <= 1e-8 or hospital_profit <= 1e-8 :
        return 10
    obj = np.log(hospital_profit - disagreement) * (1 - betas[0]) + np.log(insurer_profits[0]) * betas[0]
    return -obj


#TODO nash objective actually depends on the thing we're optimzing over e.g. dphi1/dphi2 = 0... 
def passive_disagreement(phi_vec, cost, wtp, mc):
    s = calc_s(phi_vec, cost,wtp)
    phi = np.array(phi_vec)
    mc = np.array(mc)
    # Disagreement: hospital gets only business with OTHER insurers
    return np.sum(s*(phi-mc)) - s[0]*(phi[0]-mc[0])


def recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=True):
    n = len(phi_vec)
    if analytic_base and n <= 3:
        # For 1- or 2-firm base: use all analytic phi (active beliefs by default here)
        sub_phi = np.array([analytic_phi(cost, wtp, n-1, belief='active')] * n)
        hospital_profit, _, _, _ = calc_profits(sub_phi, cost, wtp, mc[:n])
        return hospital_profit
    if n == 1:
        # (should never reach here if analytic_base is True... but as fallback)
        phi_init = [analytic_phi(cost, wtp, 1, belief='active')]
        result = minimize(lambda phi: nash_obj(phi, cost, wtp, [mc[0]], [betas[0]]),
                          phi_init, method='Nelder-Mead', options={'disp': False})
        phi_star = result.x
        hosp_profit, _, _, _ = calc_profits(phi_star, cost, wtp, [mc[0]])
        return hosp_profit
    # n-1 agents: recursively solve Nash for subgame (all but the first)
    keep = list(range(1, n))
    phi_sub = np.array([phi_vec[j] for j in keep])
    mc_sub = np.array([mc[j] for j in keep])
    betas_sub = [betas[j] for j in keep]
    phi_sub_sol = solve_nash(phi_sub, cost, wtp, mc_sub, betas_sub, analytic_base=analytic_base)
    hospital_profit, _, _, _ = calc_profits(phi_sub_sol, cost, wtp, mc_sub)
    return hospital_profit


def nash_obj(phi_vec, cost, wtp, mc, betas, active=True,analytic_base=True,phi_tmp_prev=0):
    disagreement = 0
    n = len(phi_vec)
    if n == 1:
        return disagreement
    if active:
        disagreement = recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=analytic_base)
    else:
        phi_vec_tmp  = phi_vec.copy()
        phi_vec_tmp[0] = phi_tmp_prev #keep the current firm phi fixed...
        disagreement = passive_disagreement(phi_vec_tmp, cost, wtp, mc)

    return nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement)


def solve_nash(phi_init, cost, wtp, mc, betas=None, active=False, maxiter=100, tol=1e-7,analytic_base=True, verbose=False):
    """
    Contraction mapping for n-firm Nash-in-Nash bargaining.
    phi_init: initial guess vector
    mc, betas are vectors
    """
    n = len(phi_init)
    if betas is None:
        betas = [0.5] * n
    phi = np.array(phi_init, dtype=float)
    phi_prev = phi.copy()

    for it in range(maxiter):
        phi_prev[:] = phi[:]
        for k in range(n):
            # Minimize over a SCALAR phi_k, holding all other phi fixed.
            def helper(phi_k_scalar):
                phi_tmp  = phi_prev.copy()
                phi_tmp_prev = phi_tmp[0]#phi_prev[k].copy()
                phi_tmp[k] = phi_k_scalar[0]  # phi_k_scalar is a 1-element array/lst
                # Roll so agent k is in 0 position for nash_obj
                phi_tmp_roll = np.roll(phi_tmp, -k)
                mc_roll = np.roll(mc, -k)
                betas_roll = np.roll(betas, -k)
                if active:
                    return nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll,active=True)
                else:
                    return nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll,active=False,
                                    phi_tmp_prev=phi_tmp_prev) #likely need to have 2 objectives...
            res = minimize(helper, [phi[k]], method='Nelder-Mead', options={'disp': False})
            phi[k] = res.x[0]
        diff = np.max(np.abs(phi - phi_prev))
        phi_prev = phi.copy()

        if verbose:
            print(f"Iter {it}, phi={phi}, diff={diff}")
        if diff < tol:
            break
    return phi

In [3]:
phi = np.array([14.4375,13.75,13.75])
lambd = 5.0  # try as needed
wtp = 25.0  # just needs to be larger than all phi+lambd
prices = calc_price(phi, lambd, wtp)
print("Prices:", prices)

Prices: [ 0.   18.75 18.75]


In [4]:
calc_price([ 0. ,  18.75 ,18.75], lambd, wtp)

array([11.25, 17.5 ,  0.  ])

In [5]:
lambd = 5.0  # try as needed
wtp = 25.0  # just needs to be larger than all phi+lambd
s = calc_s([14.4375,13.75,13.75], lambd, wtp)
print("shares:", s )

shares: [0.  0.5 0.5]


In [6]:
# --- Model run/prints ---
COST = 6
WTP = 25
MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_pass = [analytic_phi(COST, WTP, n, belief='passive')]*n
phi_passive = solve_nash(phi_init_pass, COST, WTP, np.zeros(n), [0.5]*n, active=False)
print("PASSIVE φ:", phi_passive)
hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, COST, WTP, np.zeros(n))
print("   hospital profit:", hospital_profit, "insurers:", insurer_profits, "shares:", s, "prices:", prices)
print("Analytic φ (passive):", analytic_phi(COST, WTP, n, "passive"))

PASSIVE φ: [13.48353046 13.48353046 13.48353046]
   hospital profit: 13.483530461543836 insurers: [0.66666667 0.66666667 0.66666667] shares: [0.33333333 0.33333333 0.33333333] prices: [15.48353046 15.48353046 15.48353046]
Analytic φ (passive): 13.5


In [7]:
# --- Model run/prints ---
COST = 5
WTP = 26

MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_act = [analytic_phi(COST, WTP, n, belief='active')]*n
phi_active = solve_nash(phi_init_act, COST, WTP, np.zeros(n), [0.5]*n, active=True)
print("ACTIVE φ:", phi_active)
hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, COST, WTP, np.zeros(n))
print("   hospital profit:", hospital_profit_a, "insurers:", insurer_profits_a, "shares:", s_a, "prices:", prices_a)
print("Analytic φ (active):", analytic_phi(COST, WTP, n, "active"))

ACTIVE φ: [17.98901699 17.98901699 17.98901699]
   hospital profit: 17.989016990447013 insurers: [0.55555556 0.55555556 0.55555556] shares: [0.33333333 0.33333333 0.33333333] prices: [19.65568366 19.65568366 19.65568366]
Analytic φ (active): 18.0


In [8]:
def run_active_passive_tables():
    import numpy as np
    import pandas as pd

    COST = 5
    l_fixed = 5
    v_vals = np.arange(24, 28)  # 24 to 28 inclusive
    n = 3
    MC = np.array([0, 0, 0])
    betas = [0.5]*n

    # --- Table 1: Vary v, fix λ=5 ---
    headers = ["WTP (v)", "phi (active)", "hospital_profit_a", "insurer_profits_a", "shares_a", "prices_a"]
    table_active_v = []

    headers2 = ["WTP (v)", "phi (passive)", "hospital_profit", "insurer_profits", "shares", "prices"]
    table_passive_v = []

    for v in v_vals:
        phi_init_act = [analytic_phi(l_fixed, v, n, belief='active')]*n
        phi_active = solve_nash(phi_init_act, l_fixed, v, MC, betas, active=True)
        hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, l_fixed, v, MC)
        # Take only first value from lists:
        row = [v, 
               np.round(phi_active[0],2),
               np.round(hospital_profit_a,2),
               np.round(insurer_profits_a[0],2),   # <-- only first item
               np.round(s_a[0],2),                 # <-- only first item
               np.round(prices_a[0],2)]            # <-- only first item
        table_active_v.append(row)

        phi_init_pass = [analytic_phi(l_fixed, v, n, belief='passive')]*n
        phi_passive = solve_nash(phi_init_pass, l_fixed, v, MC, betas, active=False)
        hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, l_fixed, v, MC)
        row2 = [v,
                np.round(phi_passive[0],2),
                np.round(hospital_profit,2),
                np.round(insurer_profits[0],2),    # <-- only first item
                np.round(s[0],2),                  # <-- only first item
                np.round(prices[0],2)]             # <-- only first item
        table_passive_v.append(row2)

    # --- Table 2: Vary λ, fix v=25 ---
    lambdas = np.arange(4, 8)  # 4 to 8 inclusive

    headers3 = ["COST (λ)", "phi (active)", "hospital_profit_a", "insurer_profits_a", "shares_a", "prices_a"]
    table_active_l = []

    headers4 = ["COST (λ)", "phi (passive)", "hospital_profit", "insurer_profits", "shares", "prices"]
    table_passive_l = []

    for lmbd in lambdas:
        phi_init_act = [analytic_phi(lmbd, 25, n, belief='active')]*n
        phi_active = solve_nash(phi_init_act, lmbd, 25, MC, betas, active=True)
        hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, lmbd, 25, MC)
        row = [lmbd,
               np.round(phi_active[0],2),
               np.round(hospital_profit_a,2),
               np.round(insurer_profits_a[0],2),   # <-- only first item
               np.round(s_a[0],2),                 # <-- only first item
               np.round(prices_a[0],2)]            # <-- only first item
        table_active_l.append(row)

        phi_init_pass = [analytic_phi(lmbd, 25, n, belief='passive')]*n
        phi_passive = solve_nash(phi_init_pass, lmbd, 25, MC, betas, active=False)
        hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, lmbd, 25, MC)
        row2 = [lmbd,
                np.round(phi_passive[0],2),
                np.round(hospital_profit,2),
                np.round(insurer_profits[0],2),    # <-- only first item
                np.round(s[0],2),                  # <-- only first item
                np.round(prices[0],2)]             # <-- only first item
        table_passive_l.append(row2)

    # --- PRINT TABLES ---
    df1 = pd.DataFrame(table_active_v, columns=headers)
    df2 = pd.DataFrame(table_passive_v, columns=headers2)
    df3 = pd.DataFrame(table_active_l, columns=headers3)
    df4 = pd.DataFrame(table_passive_l, columns=headers4)

    with pd.ExcelWriter('hotelling_results.xlsx') as writer:
        df1_sheet = "Active_WTP"
        df2_sheet = "Passive_WTP"
        df3_sheet = "Active_COST"
        df4_sheet = "Passive_COST"

        df1_title = pd.DataFrame([["=== ACTIVE: Varying WTP (v) with λ=5 ==="]])
        df1_title.to_excel(writer, sheet_name=df1_sheet, header=False, index=False, startrow=0)
        df1.to_excel(writer, sheet_name=df1_sheet, index=False, startrow=2)

        df2_title = pd.DataFrame([["=== PASSIVE: Varying WTP (v) with λ=5 ==="]])
        df2_title.to_excel(writer, sheet_name=df2_sheet, header=False, index=False, startrow=0)
        df2.to_excel(writer, sheet_name=df2_sheet, index=False, startrow=2)

        df3_title = pd.DataFrame([["=== ACTIVE: Varying COST (λ) with v=25 ==="]])
        df3_title.to_excel(writer, sheet_name=df3_sheet, header=False, index=False, startrow=0)
        df3.to_excel(writer, sheet_name=df3_sheet, index=False, startrow=2)

        df4_title = pd.DataFrame([["=== PASSIVE: Varying COST (λ) with v=25 ==="]])
        df4_title.to_excel(writer, sheet_name=df4_sheet, header=False, index=False, startrow=0)
        df4.to_excel(writer, sheet_name=df4_sheet, index=False, startrow=2)

    print("Results saved to hotelling_results.xlsx")

    # Only print first row of each DataFrame (as requested)
    print(df1.head(1))
    print(df2.head(1))
    print(df3.head(1))
    print(df4.head(1))

    return df1,df2,df3,df4
# Run
run_active_passive_tables()

Results saved to hotelling_results.xlsx
   WTP (v)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
0       24         16.99              16.99               0.56      0.33   

   prices_a  
0     18.66  
   WTP (v)  phi (passive)  hospital_profit  insurer_profits  shares  prices
0       24          11.24            11.24             0.56    0.33    12.9
   COST (λ)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
0         4         16.49              16.49               0.44      0.33   

   prices_a  
0     17.82  
   COST (λ)  phi (passive)  hospital_profit  insurer_profits  shares  prices
0         4           8.99             8.99             0.44    0.33   10.32


(   WTP (v)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
 0       24         16.99              16.99               0.56      0.33   
 1       25         17.49              17.49               0.56      0.33   
 2       26         17.99              17.99               0.56      0.33   
 3       27         18.49              18.49               0.56      0.33   
 
    prices_a  
 0     18.66  
 1     19.16  
 2     19.66  
 3     20.16  ,
    WTP (v)  phi (passive)  hospital_profit  insurer_profits  shares  prices
 0       24          11.24            11.24             0.56    0.33    12.9
 1       25          11.24            11.24             0.56    0.33    12.9
 2       26          11.24            11.24             0.56    0.33    12.9
 3       27          11.24            11.24             0.56    0.33    12.9,
    COST (λ)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
 0         4         16.49              16.49               0.44      0.3